In [53]:
# import dependencies
import gspread
import json
import geojson
import pandas as pd
import uuid
from pathlib import Path
from geographiclib.geodesic import Geodesic
import openrouteservice
from geopy.geocoders import Nominatim
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from time import sleep

The following all export their respective Google sheet to JSON / CSV for the website, and updates the Google sheet if necessary.

- [Overview](#overview) - Parses `photo` and `zoomBounds` if necessary.
- [Activity](#activity) - Adds ID's. Generates latitude and longitude (`lat/lng`) coordinates, if activity lacks them.
- [Location](#location) - Adds ID's. Generates latitude and longitude (`lat/lng`) coordinates, if location lacks them.
- [Routes](#routes) - Add's ID's. Generates `lat/lng` coordinates, if missing. Adds hikes with JSON `route_path`. Generates Great Circle paths for airplane flights. Generates automobile routes (can be [manually set](#manual-coordinates)), and will simulate a train route using automobile routes.

To update:

**Overview** - Mostly manual. Can set `id` with `=ROW()-1`. `name` prominently shows up in popup and hover tooltip. Can use `./lat_lng_geopy.ipynb` for generating `lat` and `lng`, but it's probably easier to snag coordinates off of Google Maps. Haven't used `start_date` and `end_date` yet. `photo_album` and `photos` (structure as a python list) must contain the EXACT folder and filenames - capitalization matters. `description` and `notes` are optional and will display in the popup. `importance` is subjective and scales the marker size. Setting `visit_type` to "school" and `home` to TRUE styles the Waypoint marker border and adds popup icons. `zoomBounds` and `zoomLevel` are options for zooming from the popup.

**Location** -  `location_id` auto-generates. `name` prominently shows up in popup and hover tooltip. The `location` field is used to generate `lat` and `lng` if empty. `description` and `notes` are optional and will display in the popup.

**Activity** - `activity_id` auto-generates. `name` prominently shows up in popup and hover tooltip. The `location` field is used to generate `lat` and `lng` if empty. `activity_type` must match the icon list in `../docs/static/js/overlays.js`. `route_path` enables the Routes layer on zoom, and auto-populates the Routes sheet for hiking. `description` and `notes` are optional and will display in the popup. `photo_album` and `photos` (structure as a python list) must contain the EXACT folder and filenames - capitalization matters.

**Routes** - `route_id` auto-generates. The `start_location` and `end_location` fileds are used to auto-generate routes. `transport_mode` determines route generation and display and only accepts the following values: `auto`, `plane`, `boat`, `train`, and `hike`. `filename` auto-generates for `auto`, `plane` and `hike` routes.
- **hike** - Automatically added from the Activity sheet if there is a `route_path`. Can also be manually added. Use an [Overpass Turbo](https://overpass-turbo.eu/) query to find trails.
- **plane** - Automatically generates Great Circle routes that cross the international date line. Recommend ensuring that `start_location` and `end_location` are airports.
- **auto** - Automatically generated, but amy be finicky about geocoding `start_location` and `end_location`. Can manually set start and end `lat/lng` [here](#manual-coordinates).
- **train** - Muse be manually added. Use an [Overpass Turbo](https://overpass-turbo.eu/) query to find train routes. Use `./trim_geojson.ipynb` to edit GeoJSON files. The program will generate an automobile route styled to resemble a train route if one is not manually added.
- **boat** - Muse be manually added. Use an [Overpass Turbo](https://overpass-turbo.eu/) query to find ferry routes, or draw with [geojson.io](https://geojson.io/). Use `./trim_geojson.ipynb` to edit Overpass Turbo routes.
- N.B., I try to store Overpass Turbo routes in `./overpass_routes.ipynb` for future reference.

In [54]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("api_keys.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key("12L4EkdRqaQ_e42fGHWaTmgCeqQrNgjTfoeAEc5AB6tw")

## Overview

[Back to Top](#)

In [55]:
# get data from google sheet
overview_sheet = spreadsheet.worksheet("Overview")
overview_data = overview_sheet.get_all_records()

In [56]:
# convert photo lists stored as strings into actual lists
def parse_photo_list(value):
    if isinstance(value, str) and value.strip():
        return [item.strip() for item in value.split(",")]
    return []

In [57]:
# parse zoomBounds if it's stored as a string
def parse_zoom_bounds(value):
    import json
    if isinstance(value, str) and value.strip():
        try:
            # parse the string as JSON
            return json.loads(value)
        except json.JSONDecodeError:
            print(f"Invalid zoomBounds format: {value}")
    return None  # if parsing fails or value is empty


In [58]:
# process data
for entry in overview_data:
    # parse and clean up photos list
    raw_photos = entry.get("photos", "")
    entry["photos"] = [photo.strip('[]"') for photo in parse_photo_list(raw_photos)]

    # parse zoomBounds into a proper list
    raw_zoom_bounds = entry.get("zoomBounds", "")
    entry["zoomBounds"] = parse_zoom_bounds(raw_zoom_bounds)

In [59]:
# save as JSON for JavaScript map
with open("../docs/resources/data/overview.json", "w") as file:
    json.dump(overview_data, file, indent=2)

print("Travel data successfully saved!")

Travel data successfully saved!


## Activity

[Back to Top](#)

In [60]:
# load Activity data
activity_sheet = spreadsheet.worksheet("Activity")
activity_data = pd.DataFrame(activity_sheet.get_all_records())

# add activity IDs
activity_data["activity_id"] = [
    str(uuid.uuid4()) if pd.isna(id) or id == "" else id
    for id in activity_data.get("activity_id", [])
]

In [61]:
# geocode locations missing lat/lng
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# cache for geocoding results, to avoid repeated requests / rate limits
try:
    with open("./geocode_cache.json", "r") as cache_file:
        geocode_cache = json.load(cache_file)
except FileNotFoundError:
    geocode_cache = {}


def geocode_activity_location(location_name):
    # check cache first
    if location_name in geocode_cache:
        return geocode_cache[location_name]["lat"], geocode_cache[location_name]["lng"]
    try:
        location = geolocator.geocode(location_name)
        if location:
            lat_lng = {"lat": location.latitude, "lng": location.longitude}
            geocode_cache[location_name] = lat_lng  # cache result
            return lat_lng["lat"], lat_lng["lng"]
        else:
            print(f"No lat/lng found for location: {location_name}")
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {location_name}: {e}")
        return pd.Series([None, None])


# geocode if lat/lng are missing
for index, row in activity_data.iterrows():
    if not row["lat"] or not row["lng"] or pd.isna(row["lat"]) or pd.isna(row["lng"]):
        lat, lng = geocode_activity_location(row["location"])
        print(f"Geocoding {row['location']}...")
        activity_data.at[index, "lat"] = lat
        activity_data.at[index, "lng"] = lng

        # save progress every 5 requests
        if index % 5 == 0:
            with open("./geocode_cache.json", "w") as cache_file:
                json.dump(geocode_cache, cache_file)

        # for Nominatim rate limits
        # sleep(1)

# save final cache
with open("./geocode_cache.json", "w") as cache_file:
    json.dump(geocode_cache, cache_file)

In [62]:
# save updated data to csv for JavaScript map
activity_data.to_csv("../docs/resources/data/activity.csv", index=False)

print("Activity data successfully saved to csv!")

Activity data successfully saved to csv!


In [63]:
# upload updated data to Google Sheets

# replace NaN or None values with empty strings for Google Sheets compatibility
activity_data = activity_data.fillna("")

# convert df to lists of lists
activity_data_list = [
    activity_data.columns.values.tolist()
] + activity_data.values.tolist()

# upload Activity sheet
activity_sheet = spreadsheet.worksheet("Activity")
try:
    activity_sheet.clear()  # clear existing data
    activity_sheet.update(values=activity_data_list, range_name="A1")  # upload new data
except Exception as e:
    print(f"Error updating Activity sheet: {e}")

print("Activity data successfully uploaded to Google Sheets!")

Activity data successfully uploaded to Google Sheets!


## Location

[Back to top](#)

In [64]:
# load Location data
location_sheet = spreadsheet.worksheet("Location")
location_data = pd.DataFrame(location_sheet.get_all_records())

# add location IDs
location_data["location_id"] = [
    str(uuid.uuid4()) if pd.isna(id) or id == "" else id
    for id in location_data.get("location_id", [])
]

In [65]:
# geocode if lat/lng are missing
for index, row in location_data.iterrows():
    if not row["lat"] or not row["lng"] or pd.isna(row["lat"]) or pd.isna(row["lng"]):
        lat, lng = geocode_activity_location(row["location"])
        print(f"Geocoding {row['location']}...")
        location_data.at[index, "lat"] = lat
        location_data.at[index, "lng"] = lng

        # save progress every 5 requests
        if index % 5 == 0:
            with open("./geocode_cache.json", "w") as cache_file:
                json.dump(geocode_cache, cache_file)

        # for Nominatim rate limits
        # sleep(1)

# save final cache
with open("./geocode_cache.json", "w") as cache_file:
    json.dump(geocode_cache, cache_file)

In [66]:
# save updated data to csv for JavaScript map
location_data.to_csv("../docs/resources/data/locations.csv", index=False)

print("Location data successfully saved to csv!")

Location data successfully saved to csv!


In [67]:
# upload updated data to Google Sheets

# replace NaN or None values with empty strings for Google Sheets compatibility
location_data = location_data.fillna("")

# convert df to lists of lists
location_data_list = [
    location_data.columns.values.tolist()
] + location_data.values.tolist()

# upload Location sheet
location_sheet = spreadsheet.worksheet("Location")
try:
    location_sheet.clear()  # clear existing data
    location_sheet.update(values=location_data_list, range_name="A1")  # upload new data
except Exception as e:
    print(f"Error updating Location sheet: {e}")

print("Location data successfully uploaded to Google Sheets!")

Location data successfully uploaded to Google Sheets!


## Routes

[Back to top](#)

In [68]:
# load route data
route_sheet = spreadsheet.worksheet("Routes")
route_data = pd.DataFrame(route_sheet.get_all_records())

# cache for geocoding results, to avoid repeated requests / rate limits
try:
    with open("./geocode_cache_routes.json", "r") as cache_file:
        geocode_cache_routes = json.load(cache_file)
except FileNotFoundError:
    geocode_cache_routes = {}

#### Add Hike routes if in Activity and with a geojson

In [69]:
# add Activity geojson to Routes
for index, row in activity_data.iterrows():
    # if hiking activities with a route_path not in Routes.filename
    if (
        row["activity_type"] == "hiking"  # Activity type is hiking
        and row["route_path"]  # Activity has a route_path
        and row["route_path"] not in route_data["filename"].values  # route_path not in Routes.filename
    ):
        # add new route to Routes
        new_route = {
            "start_location": row["name"],  # Activity.name
            "end_location": row["name"],  # Activity.name
            "transport_mode": "hike",  # Fixed value
            "filename": row["route_path"],  # Activity.route_path
        }
        route_data = pd.concat([route_data, pd.DataFrame([new_route])], ignore_index=True)


In [70]:
# add route IDs
route_data["route_id"] = [
    str(uuid.uuid4()) if pd.isna(id) or id == "" else id
    for id in route_data.get("route_id", [])
]

#### Generates Airplane great circles

In [71]:
# airplane routes

# calculate great circle route
def calculate_great_circle(start_coords, end_coords, num_points=100):
    geod = Geodesic.WGS84
    line = geod.InverseLine(start_coords[0], start_coords[1], end_coords[0], end_coords[1])
    
    # intermediate points along the great circle
    points = []
    for i in range(num_points + 1):
        s = i * line.s13 / num_points
        position = line.Position(s)
        lon, lat = position["lon2"], position["lat2"]
        
        # handle crossing the International Date Line
        if points and abs(lon - points[-1][0]) > 180:
            if lon > 0:
                lon -= 360  # shift longitude from +180 to -180
            else:
                lon += 360  # shift longitude from -180 to +180
        
        points.append((lon, lat))  # (longitude, latitude)
    
    return points

# save as GeoJSON
def save_great_circle_as_geojson(route_coords, output_file):
    feature = geojson.Feature(
        geometry=geojson.LineString(route_coords),
        properties={"transport_mode": "plane"}
    )
    feature_collection = geojson.FeatureCollection([feature])
    with open(output_file, "w") as f:
        geojson.dump(feature_collection, f)
    print(f"Great circle route saved to {output_file}")

# process routes from Google Sheet
def process_plane_routes_from_sheet(route_data):
    for index, row in route_data.iterrows():
        transport_mode = row["transport_mode"].lower()
        start_location = row["start_location"]
        end_location = row["end_location"]
        filename = row.get("filename", "").strip() 

        # check if "plane" and filename is empty
        if transport_mode == "plane" and not filename:
            print(f"Processing plane route: {start_location} -> {end_location}")

            # check if start and end locations are in cache
            start_coords = geocode_cache_routes.get(start_location)
            end_coords = geocode_cache_routes.get(end_location)

            # if not in cache, geocode the locations, add location to cache
            if not start_coords:
                geolocator = Nominatim(user_agent="geoapi", timeout=10)
                location = geolocator.geocode(start_location)
                if location:
                    start_coords = (location.latitude, location.longitude)
                    geocode_cache_routes[start_location] = start_coords  # cache result
                else:
                    print(f"Failed to geocode start location: {start_location}")
                    continue

            if not end_coords:
                geolocator = Nominatim(user_agent="geoapi", timeout=10)
                location = geolocator.geocode(end_location)
                if location:
                    end_coords = (location.latitude, location.longitude)
                    geocode_cache_routes[end_location] = end_coords  # cache result
                else:
                    print(f"Failed to geocode end location: {end_location}")
                    continue

            # get great circle route
            route_coords = calculate_great_circle(start_coords, end_coords)

            # save as GeoJSON
            filename = f"great_circle_route_{index}.geojson"
            save_great_circle_as_geojson(route_coords, f"../docs/resources/geojson/{filename}")

            # update route_data with geojson filename
            route_data.at[index, "filename"] = filename
            print(f"Updated Google Sheet with filename: {filename}")

    # save updated cache to file
    with open("./geocode_cache_routes.json", "w") as cache_file:
        json.dump(geocode_cache_routes, cache_file)

# call function to process routes
process_plane_routes_from_sheet(route_data)

#### Generates Automobile routes, will fake a Train route with an auto route

In [72]:
# connect to OpenRouteService API for automobile routes
with open("api_keys.json") as f:
    api_keys = json.load(f)
ors_client = openrouteservice.Client(key=api_keys["openrouteservice"])

In [73]:
# route function
def fetch_route(start_coords, end_coords, transport_mode):
    try:
        profile = {
            "auto": "driving-car",
            # ORS doesn't support trains - find alterative API or method
            "train": "driving-car",
        }.get(transport_mode, "driving-car")

        # request route from ors api
        route = ors_client.directions(
            coordinates=[start_coords, end_coords], profile=profile, format="geojson"
        )
        return route
    except Exception as e:
        print(f"Failed to fetch route for mode {transport_mode}: {e}")
        return None

In [74]:
# geocode coordinates a location
def geocode_route_location(location):
    # check cache first
    if location in geocode_cache_routes:
        return geocode_cache_routes[location]
    try:
        geocode_response = ors_client.pelias_search(text=location)
        if geocode_response["features"]:
            coords = geocode_response["features"][0]["geometry"]["coordinates"]
            geocode_cache_routes[location] = (coords[0], coords[1])  # cache result
            return coords[0], coords[1]  # return (lon, lat)
    except Exception as e:
        print(f"Failed to geocode location {location}: {e}")
    return None, None

## Manual Coordinates

[Back to top](#)

In the following dictionary, manually set the start_location and end_location coordinates. The name must match the spreadsheet location exactly. 

#### <b>Format: (longitude, latitude)</b>

In [75]:
# manually set lng/lat here
locations = {
    "Fish River Canyon": (17.614818, -27.589350),  # Fish River Canyon
    "N7 to Noordoewer Border Post": (17.830023, -29.020174),
    "N7 again to Noordoewer Border Post": (17.697956, -28.843831),
    "Border Posts - Vioolsdrift": (17.626150, -28.770706),
    "Sesriem": (15.803785, -24.491391),  # Sesriem
    "Sesriem Canyon": (15.803785, -24.491391),  # Sesriem Canyon
    "Kokerboomwoud": (18.241756, -26.481496),  # Kokerboomwoud
    "Deadvlei": (15.322147, -24.729733),  # Deadvlei
    "Dune 45": (15.471035, -24.723004),  # Dune 45
    "Giant's Playground": (18.270635, -26.464834),  # Giant's Playground
    "Keetmanshoop": (18.138505, -26.585550),  # Keetmanshoop
    "Helmeringhausen": (16.821412, -25.889119),  # Helmeringhausen
}

In [76]:
# process GeoJSON routes and save to file
for index, row in route_data.iterrows():
    # skip extant filenames
    if pd.notna(row["filename"]) and row["filename"].strip() != "":
        # print(f"Skipping route {row['route_id']} as it already has a filename: {row['filename']}")
        continue

    # check if start_location and end_location exist in the manually set locations, default to geocode
    start_coords = locations.get(row["start_location"], geocode_route_location(row["start_location"]))
    end_coords = locations.get(row["end_location"], geocode_route_location(row["end_location"]))

    # if geocoding was successful, fetch the route
    if None not in start_coords and None not in end_coords:
        route = fetch_route(start_coords, end_coords, row["transport_mode"])

        # if route, save to GeoJSON file, update route_data df
        if route:
            geojson_filename = f"{row['route_id']}.geojson"
            geojson_path = Path("../docs/resources/geojson/") / geojson_filename

            with open(geojson_path, "w") as f:
                json.dump(route, f, indent=4)

            route_data.at[index, "filename"] = geojson_filename
            print(
                f"Generated GeoJSON for route {row['start_location']} to {row['end_location']}, and saved to {geojson_path}"
            )

# save updated cache to file
with open("./geocode_cache_routes.json", "w") as cache_file:
    json.dump(geocode_cache_routes, cache_file)

#### Save to csv, updates google sheets

In [77]:
# save route data to csv
route_data.to_csv("../docs/resources/data/routes.csv", index=False)
print("Route data successfully saved to csv!")

Route data successfully saved to csv!


In [78]:
# upload df back to google sheets
set_with_dataframe(route_sheet, route_data)
print("Route data successfully uploaded to Google Sheets!")

Route data successfully uploaded to Google Sheets!
